# Using computer vision techniques to enhance product search and discovery

- Develop an image-based search feature for visually similar products.
- Implement automatic product categorization based on image features. (not done)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from scipy.spatial.distance import cosine
from bs4 import BeautifulSoup
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import sqlite3
import time
import os
import requests

model = ResNet50(weights='resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                 include_top=False, pooling='avg')


def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    features = model.predict(x[None, ...])
    return features

In [ ]:
def create_table():
    # Connect to the SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('feature_database.db')
    cursor = conn.cursor()

    # Create the table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS features (
        id INTEGER PRIMARY KEY,
        filename TEXT,
        features BLOB
    )
    ''')
    conn.commit()
    conn.close()


create_table()

The code below using the image link to download the images into the amazon_images folder. If the link is unusable, it insteads webscrapes the Amazon website using the product link. (Running the code below takes about an hour)


In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}
images = amazon_df[["img_link", "product_link"]]
n = len(images)
temp = 0
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)

for i in range(1, n+1):
    response = requests.get(images.iloc[i-1]["img_link"])
    if response.status_code == 200:
        with open("amazon_images/image_" + str(temp) + ".jpg", "wb") as file:
            file.write(response.content)
        temp += 1
    else:
        response = requests.get(images.iloc[i-1]["product_link"])
        product_link = images.iloc[i-1]["product_link"]
        driver.get(product_link)
        time.sleep(3)

        try:
            image_tag = driver.find_element(By.ID, 'landingImage')
            soup = BeautifulSoup(response.content, 'html.parser')
            image_url = image_tag.get_attribute('src')

            if image_url:

                img_response = requests.get(image_url)
                img_response.raise_for_status()
                with open("amazon_images/image_" + str(temp) + ".jpg", "wb") as file:
                    file.write(img_response.content)
                temp += 1
        except Exception as e:
            print("file number" + str(i) + "did not print")

file number193did not print
file number297did not print
file number319did not print
file number635did not print
file number718did not print
file number825did not print
file number828did not print
file number866did not print
file number998did not print
file number1028did not print
file number1081did not print
file number1153did not print
file number1159did not print
file number1217did not print
file number1234did not print
file number1259did not print
file number1290did not print
file number1310did not print
file number1360did not print
file number1389did not print
file number1437did not print


In [ ]:
def insert_features(id, filename, features):
    conn = sqlite3.connect('feature_database.db')
    cursor = conn.cursor()

    # Convert features to binary format
    features_blob = features.tobytes()

    cursor.execute('''
    INSERT INTO features (id, filename, features) VALUES (?, ?, ?)
    ''', (id, filename, features_blob))

    conn.commit()
    conn.close()


folder_path = 'amazon_images/'
temp = 0

for img in os.listdir(folder_path):
    try:
        img_path = os.path.join(folder_path, img)
        img_features = extract_features(img_path)
        insert_features(temp, img, img_features)
        temp += 1
    except:
        print(str(img) + " was unable to be inserted")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━

 17%|█▋        | 329/1977 [40:15:48<201:41:03, 440.57s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━

In [ ]:
def find_similar_features(query_features):
    conn = sqlite3.connect('feature_database.db')
    cursor = conn.cursor()

    cursor.execute('SELECT id, features FROM features')
    rows = cursor.fetchall()

    similar_items = []
    for row in rows:
        id, features_blob = row
        features = np.frombuffer(features_blob, dtype=np.float32)
        distance = cosine(query_features[0], features)
        similar_items.append([id, distance])

    similar_items.sort(key=lambda x: x[1])
    conn.close()
    return similar_items


query_features = extract_features('test_images/wayonna.png')
similar_items = find_similar_features(query_features)
similar_items

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 817ms/step


[[818, 0.14651812818105503],
 [1044, 0.14651812818105503],
 [1068, 0.16831125126672986],
 [6, 0.19165923214346048],
 [330, 0.1916746106669316],
 [462, 0.19251936481292176],
 [520, 0.19482023697325124],
 [815, 0.19577942689078265],
 [561, 0.21211989356558558],
 [116, 0.22168582094643463],
 [368, 0.2226647198804791],
 [1195, 0.2351429796557345],
 [213, 0.24598012987063134],
 [262, 0.25396970585005496],
 [1073, 0.2598501435798005],
 [56, 0.2603783488468595],
 [697, 0.2714103585700901],
 [376, 0.27213362259048046],
 [591, 0.2775149023923391],
 [1260, 0.2775149023923391],
 [14, 0.2783544372288135],
 [446, 0.2865633712786845],
 [831, 0.29056262578604053],
 [1280, 0.2942802873823781],
 [652, 0.29457326929177186],
 [408, 0.2963183585515159],
 [828, 0.2963183585515159],
 [989, 0.2972167631138155],
 [1232, 0.2972167631138155],
 [485, 0.29815632017421956],
 [1199, 0.29815632017421956],
 [481, 0.30131655614441255],
 [834, 0.3022545860892124],
 [1074, 0.30330211642801785],
 [724, 0.3087780340362884

In [ ]:
def get_features(id):
    conn = sqlite3.connect('feature_database.db')
    cursor = conn.cursor()

    cursor.execute('SELECT features FROM features WHERE id=?', (id,))
    result = cursor.fetchone()

    conn.close()

    if result:
        features_blob = result[0]
        features = np.frombuffer(features_blob, dtype=np.float32)
        return features
    else:
        return None


features = get_features(similar_items[0][0])
print(features)

[0.04215956 0.30813807 0.08206543 ... 0.         0.8073022  1.1324998 ]


In [ ]:
def get_image(id):
    conn = sqlite3.connect('feature_database.db')
    cursor = conn.cursor()

    cursor.execute('SELECT filename, features FROM features WHERE id=?', (id,))
    result = cursor.fetchone()

    conn.close()

    if result:
        filename, features_blob = result
        features = np.frombuffer(features_blob, dtype=np.float32)

        image_path = f'amazon_images/{filename}'  # Update with actual path
        img = Image.open(image_path)

        return img, features
    else:
        return None, None


image, features = get_image(similar_items[0][0])
if image:
    image.show()

Python(10154) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10156) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
